In [38]:
import pandas as pd
import nbformat
import plotly

import plotly.graph_objects as go
import pandas_datareader.data as web
import datetime

In [39]:
data = {
    "Maturity": [0.5, 1, 2, 5, 10],
    "Market Rate (%)": [3.50, 3.75, 4.00, 4.50, 5.00],
    "Price": [98.25, 96.00, 92.00, 80.00, 60.00],
    # "Instrument": ["Deposit Rate", "1Y Treasury Bill", "Treasury Bond", "Swap Rate", "Swap Rate"]
}

df = pd.DataFrame(data)
df

,Maturity,Market Rate (%),Price
0,0.5,3.50,98.25
1,1.0,3.75,96.00
2,2.0,4.00,92.00
3,5.0,4.50,80.00
4,10.0,5.00,60.00


#### constructing a zero coupon curve (bootstrappping)

zero-coupon bond (ZCB) current price = FV / [ (1+spot_rate_for_maturity_t)**t ]

In [40]:
def get_discount_factor(rate, t):
    return 1/((1 + rate) ** t)

def spot_rate( price, face_value=100, maturity=1):

    return ((face_value / price) ** (1 / maturity)) - 1


In [41]:
yields  = [spot_rate(price, 100, maturity) * 100 for price, maturity in zip(df["Price"], df["Maturity"])]
df["Spot Rate (%)"]= yields 

In [42]:
# fig.show(renderer="browser")  


#### derive forward curve from spot curve

In [43]:
# def forward_rate(P_t, P_T, FV=100, t=1, T=2): #future estimate from t to T 

#     S_t = spot_rate(P_t, FV, t)
#     S_T = spot_rate(P_T, FV, T)
#     return (( ((1 + S_T)**T) / ((1 + S_t)**t) ) ** (1/(T-t))  ) -1

def forward_rate(S_t, S_T,  t=1, T=2): #future estimate from t to T 

     return (( ((1 + S_T)**T) / ((1 + S_t)**t) ) ** (1/(T-t))  ) -1



In [44]:
forward_rates = []
maturities_forward = [] 

for i in range(len(df) - 1):
    t = df["Maturity"][i]
    T = df["Maturity"][i + 1]
    S_t = df["Spot Rate (%)"][i] / 100  
    S_T = df["Spot Rate (%)"][i + 1] / 100 
    
    f_tT = forward_rate(S_t, S_T, t, T) * 100  
    forward_rates.append(f_tT)
    maturities_forward.append((t + T) / 2)  # Midpoint for plotting


In [45]:

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["Maturity"], 
    y=df["Spot Rate (%)"], 
    mode='lines+markers',
    name='Zero Curve (Spot Rates)',
    line=dict(width=2),
    marker=dict(size=8)
))

fig.add_trace(go.Scatter(
    x=maturities_forward, 
    y=forward_rates, 
    mode='lines+markers',
    name='Forward Curve',
    line=dict(width=2, dash="dash"),  # Dashed line for distinction
    marker=dict(size=8, symbol="diamond")
))

fig.update_layout(
    title="Zero Yield Curve & Forward Curve",
    xaxis_title="Maturity (Years)",
    yaxis_title="Rate (%)",
    template="plotly_dark",
    hovermode="x unified"
)


fig.show()


    real-life Treasury yield data

In [46]:
start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2025, 1, 1)

df_1yr = web.DataReader('DGS1', 'fred', start, end)
df_10yr = web.DataReader('DGS10', 'fred', start, end)

df_1yr.dropna(inplace=True)
df_10yr.dropna(inplace=True)

df = df_1yr.join(df_10yr, how='inner', lsuffix='_1yr', rsuffix='_10yr')

df['Forward Rate (%)'] = (((1 + df['DGS10'] / 100) ** 10) / ((1 + df['DGS1'] / 100) ** 1)) ** (1 / (10 - 1)) - 1
df['Forward Rate (%)'] *= 100  
df

,DGS1,DGS10,Forward Rate (%)
DATE,,,
2024-01-02,4.80,3.95,3.855982
2024-01-03,4.81,3.91,3.810478
2024-01-04,4.85,3.99,3.894881
2024-01-05,4.84,4.05,3.962591
2024-01-08,4.82,4.01,3.920387
...,...,...,...
2024-12-24,4.24,4.59,4.628961
2024-12-26,4.23,4.58,4.618961
2024-12-27,4.20,4.62,4.666771


In [47]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['DGS1'],
    mode='lines',
    name='1-Year Spot Rate',
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['DGS10'],
    mode='lines',
    name='10-Year Spot Rate',
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Forward Rate (%)'],
    mode='lines',
    name='1Y-10Y Forward Rate',
    line=dict(width=2, dash='dash')
))

fig.update_layout(
    title="Treasury Spot Rates and Forward Rate",
    xaxis_title="Date",
    yaxis_title="Rate (%)",
    template="plotly_dark",
    hovermode="x unified"
)

fig.show()

* forward rate formula is exponentially weighted by S_10 &rarr; heavily influenced by the long-term rate

In [37]:
df

,DGS1,DGS10,Forward Rate (%)
DATE,,,
2024-01-02,4.80,3.95,3.855982
2024-01-03,4.81,3.91,3.810478
2024-01-04,4.85,3.99,3.894881
2024-01-05,4.84,4.05,3.962591
2024-01-08,4.82,4.01,3.920387
...,...,...,...
2024-12-24,4.24,4.59,4.628961
2024-12-26,4.23,4.58,4.618961
2024-12-27,4.20,4.62,4.666771
